In [2]:
from transformers import AutoModelForSeq2SeqLM, pipeline, AutoTokenizer
from tqdm.autonotebook import tqdm

### Load tokenizers and models

In [3]:
base_tokenizer = AutoTokenizer.from_pretrained("../models/dbart/")

In [4]:
topic_tokenizer = AutoTokenizer.from_pretrained("../models/dbart-topic/")

In [5]:
targets_tokenizer = AutoTokenizer.from_pretrained("../models/dbart-targets/")

In [6]:
aspects_tokenizer = AutoTokenizer.from_pretrained("../models/dbart-aspects/")

In [7]:
dbart = AutoModelForSeq2SeqLM.from_pretrained("../models/dbart/")

In [8]:
dbart_topic = AutoModelForSeq2SeqLM.from_pretrained("../models/dbart-topic/")

In [9]:
dbart_targets = AutoModelForSeq2SeqLM.from_pretrained("../models/dbart-targets/")

In [10]:
dbart_aspects = AutoModelForSeq2SeqLM.from_pretrained("../models/dbart-aspects/")

In [1]:
argument = """If a lawyer has worked harder to win cases easier, they should be entitled to charge how much they want, accepting clients that can afford it. You can't buy better justice, only influence it through the wit, resources, and experience of a decent lawyer who has undoubtedly worked harder to get to his position than a third rate lawyer. The rich should be able to do whatever they want with their money, as long as it stays in the boundaries of the law. If it doesn't, then that's what the lawyer is for, to help them get the best possible outcome. However, a good lawyer can only go so far. I'm also not sure how your proposed system would work correctly. Maybe, there is only 2 lawyers in a jurisdiction, a really good one and a bad one. How would a worthy opponent be chosen for either? You surely can't expect the state, defendant, or plaintiff to get a lawyer from somewhere else shipped in? That's just dumb."""

In [11]:
topic_encoded_argument = """<|TOPIC|>It's immoral that the wealthy can hire better lawyers.<|ARGUMENT|>If a lawyer has worked harder to win cases easier, they should be entitled to charge how much they want, accepting clients that can afford it. You can't buy better justice, only influence it through the wit, resources, and experience of a decent lawyer who has undoubtedly worked harder to get to his position than a third rate lawyer. The rich should be able to do whatever they want with their money, as long as it stays in the boundaries of the law. If it doesn't, then that's what the lawyer is for, to help them get the best possible outcome. However, a good lawyer can only go so far. I'm also not sure how your proposed system would work correctly. Maybe, there is only 2 lawyers in a jurisdiction, a really good one and a bad one. How would a worthy opponent be chosen for either? You surely can't expect the state, defendant, or plaintiff to get a lawyer from somewhere else shipped in? That's just dumb.<|CONCLUSION|>"""

In [12]:
targets_encoded_argument = """<|TOPIC|>It's immoral that the wealthy can hire better lawyers.<|ARGUMENT|>If a lawyer has worked harder to win cases easier, they should be entitled to charge how much they want, accepting clients that can afford it. You can't buy better justice, only influence it through the wit, resources, and experience of a decent lawyer who has undoubtedly worked harder to get to his position than a third rate lawyer. The rich should be able to do whatever they want with their money, as long as it stays in the boundaries of the law. If it doesn't, then that's what the lawyer is for, to help them get the best possible outcome. However, a good lawyer can only go so far. I'm also not sure how your proposed system would work correctly. Maybe, there is only 2 lawyers in a jurisdiction, a really good one and a bad one. How would a worthy opponent be chosen for either? You surely can't expect the state, defendant, or plaintiff to get a lawyer from somewhere else shipped in? That's just dumb.<|TARGETS|>how your proposed system, the state defendant or plaintiff to get a lawyer from somewhere else shipped in, to do whatever they want with their money, buy better justice only influence it through the wit resources and experience of a decent lawyer who has undoubtedly worked harder to get to his position than a third rate lawyer ., to charge how much they want accepting clients that can afford it .<|CONCLUSION|>"""

In [13]:
aspects_encoded_argument = """<|TOPIC|>It's immoral that the wealthy can hire better lawyers.<|ARGUMENT|>If a lawyer has worked harder to win cases easier, they should be entitled to charge how much they want, accepting clients that can afford it. You can't buy better justice, only influence it through the wit, resources, and experience of a decent lawyer who has undoubtedly worked harder to get to his position than a third rate lawyer. The rich should be able to do whatever they want with their money, as long as it stays in the boundaries of the law. If it doesn't, then that's what the lawyer is for, to help them get the best possible outcome. However, a good lawyer can only go so far. I'm also not sure how your proposed system would work correctly. Maybe, there is only 2 lawyers in a jurisdiction, a really good one and a bad one. How would a worthy opponent be chosen for either? You surely can't expect the state, defendant, or plaintiff to get a lawyer from somewhere else shipped in? That's just dumb.<|ASPECTS|>worthy opponent, boundaries of the law, buy, resources, influence, good lawyer, accepting clients, best possible outcome, win cases, better justice, money, charge<|CONCLUSION|>"""

In [14]:
def truncate_text_and_tokenize(text):
        tokens = tokenizer(text, return_tensors = "pt", truncation=True).input_ids
        num_tokens = tokens.size()[1]
        if num_tokens>=1024:
            _tokens = tokens[0][:num_tokens-3]
        else:
            _tokens = tokens[0]
        tokenized_text = tokenizer.decode(_tokens, skip_special_tokens=False, clean_up_tokenization_spaces=True)
        return (_tokens, tokenized_text)

### Generate conclusion using dbart

In [24]:
tokenizer = base_tokenizer
summarization_pipeline = pipeline("summarization", model=dbart, tokenizer=base_tokenizer)

def generate(text):
    _tokens, tokenized_text = truncate_text_and_tokenize(text)
    summarization = summarization_pipeline(tokenized_text, clean_up_tokenization_spaces=True)
    summary_text = summarization[0]["summary_text"]
    return summary_text

In [25]:
generate(argument)

'If someone can afford the legal fees, they should be entitled to the amount they want'